In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

## Variable Scoping and name scoping


### Name scoping

The **layout in tensorboard respects the name scopes** and thus creates nice layouts. Another use case for name scoping is to make **building blocks** with can be used to assemble a network.

In [3]:
tf.reset_default_graph()
with tf.name_scope('var') as v_scope:
    b1 = tf.placeholder(dtype='float32', shape=(None, 64, 64, 3), name='Input')
    #W = tf.get_variable('W', shape=(3,3,3,10)) 
    #Is not respecting the name_scope
    W = tf.Variable(tf.truncated_normal((3,3,3,10)), name='Kernels')
    a1 = tf.nn.conv2d(b1, W, (1,1,1,1), padding='SAME')
a1.name, b1.name, W.name

(u'var/Conv2D:0', u'var/Input:0', u'var/Kernels:0')

### Using name scoping as a building block

In [4]:
tf.reset_default_graph()
def conv_layer(net, shape, scope):
    with tf.name_scope(scope) as v_scope:
        kernel = tf.Variable(tf.truncated_normal(shape, dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
        conv = tf.nn.conv2d(net, kernel, [1, 1, 1, 1], padding='SAME')
        biases = tf.Variable(tf.constant(0.0, shape=[shape[3]], dtype=tf.float32),
                                 trainable=True, name='biases')
        out = tf.nn.bias_add(conv, biases)
        return tf.nn.relu(out, name=scope)

In [5]:
net = tf.placeholder(dtype='float32', shape=(None, 64, 64, 3), name='Input')
net = conv_layer(net, [3, 3, 3, 64], 'conv1')
net = conv_layer(net, [3, 3, 64, 128], 'conv2')
net = conv_layer(net, [3, 3, 128, 128], 'conv3')
writer = tf.train.SummaryWriter("/tmp/dumm/scoping", tf.get_default_graph(), 'graph.pbtxt') 

![](tb_scoping.png) 

### Variable Scope
Variable scoping is a mechanism to share the variables of (possible large) parts of a network. These shared variables are very useful e.g. for Siamese Networks. See also https://www.tensorflow.org/versions/master/how_tos/variable_scope/index.html

#### The two functions to use
* `tf.variable_scope` created the name-space or better context manager
* `tf.get_variable()` gets or newly creates variables in the name scope 

In [6]:
tf.reset_default_graph()
with tf.variable_scope('foo'):
    with tf.variable_scope('bar'):
     d = tf.get_variable('v', shape=(1,10))
d.name

u'foo/bar/v:0'

#### Reusing variables vs new variables

##### Creating new variables
The context manager is creating new variables by default. If you request a variable, which has already been created it returns an error.

In [7]:
tf.reset_default_graph()
with tf.variable_scope('var'):
    a1 = tf.get_variable('a', shape=(1))
    #This variable is used and thus this would result in an error
    #a1_1 = tf.get_variable('a', shape=(1)) 
    a2 = tf.get_variable('a2', shape=(1))    
a1.name,a2.name

(u'var/a:0', u'var/a2:0')

##### Reusing variables

Reusing variables is important when it come to share variables such as in Siamese networks

In [9]:
tf.reset_default_graph()

with tf.variable_scope('var', reuse=False):
     a1 = tf.get_variable('a', shape=(1))

with tf.variable_scope('var', reuse=True):
    a1_1 = tf.get_variable('a', shape=(1)) #This variable is reused
    #This would give an error, since that variable has not been used before
    #a2 = tf.get_variable('a2', shape=(1)) 
a1.name, a1_1.name

(u'var/a:0', u'var/a:0')

In [8]:
tf.reset_default_graph()

with tf.variable_scope('var', reuse=False) as var_context:
    a1 = tf.get_variable('a', shape=(1))
    var_context.reuse_variables()
    #Alternatively tf.get_variable_scope().reuse_variables()
    a1_1 = tf.get_variable('a', shape=(1))
a1.name,a2.name

(u'var/a:0', u'var/a2:0')